In [1]:
import dgl
import numpy as np
import torch
import networkx as nx
import os
os.environ['DGLBACKEND'] = 'pytorch'

from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from dgl.nn import NNConv, EdgeConv
from dgl.nn.pytorch import Sequential as dglSequential
from dgl.dataloading import GraphDataLoader
from torch.utils.data import Dataset
from tqdm.notebook import tqdm

## Dataloader

In [2]:
data_path = '../data/graphdataset/'

In [3]:
class GraphDataset(Dataset):
    def __init__(self, data_path, n_files=None, transform=None):

        self.fnames    = [fname for fname in os.listdir(data_path) if fname.endswith('.dgl')]
        self.data_path = data_path

        if n_files:
            self.fnames = self.fnames[n_files]

        self.transform = transform     

        self.load_graphs()


    def load_graphs(self):
        self.graphs = []

        for fname in tqdm(self.fnames, 'Reading files'):
            graphs = dgl.load_graphs(os.path.join(self.data_path, fname))[0]
            self.graphs.extend(graphs)


    def __len__(self):

        return len(self.graphs)
    

    def __getitem__(self, idx):

        x = self.graphs[idx]
        
        if self.transform:
            x = self.transform(x)
        
        return x

In [4]:
train_files = slice(10)
val_files   = slice(10, 13)

train_dataset = GraphDataset(data_path, train_files)
val_dataset   = GraphDataset(data_path, val_files)

Reading files:   0%|          | 0/10 [00:00<?, ?it/s]

Reading files:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
train_dataloader = GraphDataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader   = GraphDataLoader(val_dataset, batch_size=64, shuffle=True)
for batch in train_dataloader:
    break
batch

Graph(num_nodes=3441, num_edges=214619,
      ndata_schemes={'f': Scheme(shape=(7,), dtype=torch.float32)}
      edata_schemes={'d': Scheme(shape=(3,), dtype=torch.float32)})

## Simulated Graph
50 particelle, 7 features per particle, each node is connected to all the others so n**2 edges, each edge has 3 different features

In [ ]:
n = 50 
src = []; dst = []
for i in range(n):
    for j in range(n):
        src.append(i)
        dst.append(j)

g = dgl.graph((src, dst))

nodes = torch.randn((n, 7))
edges = torch.randn((n**2, 3))

g.ndata['feat'] = nodes
g.edata['edge_feat'] = edges


In [ ]:
def MLP(efeat, ch=[256,128,64,32]):

    mlp = nn.Sequential(
        nn.Linear(3, ch[0]),
        nn.Dropout(0.2),
        nn.ReLU(),
        
        nn.Linear(ch[0],ch[1]),
        nn.Dropout(0.2),
        nn.ReLU(),
        
        nn.Linear(ch[1],ch[2]),
        nn.Dropout(0.2),
        nn.ReLU(),

        nn.Linear(ch[2], ch[3]),
        nn.Dropout(0.2),
        nn.ReLU(),

        nn.Linear(ch[3], 7*128),
        nn.Sigmoid()
    )
    
    return mlp(efeat)

In [ ]:
conv = NNConv(7, 128, MLP)
res = conv(g, nodes, edges)
res.shape

torch.Size([50, 128])

## Encoder

In [6]:
class Encoder(nn.Module):
    
    def __init__(self, latent_space_dim, ch=[256,128,64,32]):
        super().__init__()

        self.mlp = nn.Sequential(
            nn.Linear(3, ch[0]),
            nn.Dropout(0.2),
            nn.ReLU(),
            
            nn.Linear(ch[0],ch[1]),
            nn.Dropout(0.2),
            nn.ReLU(),
            
            nn.Linear(ch[1],ch[2]),
            nn.Dropout(0.2),
            nn.ReLU(),

            nn.Linear(ch[2], ch[3]),
            nn.Dropout(0.2),
            nn.ReLU(),

            nn.Linear(ch[3], 7*128),
            nn.Sigmoid()
            )
        
        self.conv = dglSequential(
            NNConv(
                in_feats  = 7,  # number of node features
                out_feats = 128, # output number of node features
                edge_func = self.mlp),
            EdgeConv(128, 64, batch_norm=True),
            EdgeConv(64, 32, batch_norm=True),
            EdgeConv(32, latent_space_dim, batch_norm=True)
        )

    def forward(self, graph, n_feat=None):

        x = self.conv(graph, n_feat if n_feat else graph.ndata['f'], graph.edata['d'])
        return x
    

In [ ]:
encoder = Encoder(4)
encoded = encoder(batch)
encoded.shape

torch.Size([101, 4])

In [ ]:
batch.ndata['l'] = encoded

## Decoder

In [7]:
class Decoder(nn.Module):
    
    def __init__(self, latent_space_dim, n_feat=7):
        super().__init__()

        self.shared_path = dglSequential(
            EdgeConv(latent_space_dim, 32, batch_norm=True),
            EdgeConv(32, 64, batch_norm=True),
            EdgeConv(64, 128, batch_norm=True)
        )
        
        self.node_reconstruct = EdgeConv(128, n_feat)    # output are the reconstructed node features

        self.edge_reconstruct1 = dglSequential(
            EdgeConv(128, 32, batch_norm=True),
            EdgeConv(32,16, batch_norm=True),
            EdgeConv(16,8, batch_norm=True)
        )

        self.edge_reconstruct2 = dglSequential(
            EdgeConv(128, 32, batch_norm=True),
            EdgeConv(32,16, batch_norm=True),
            EdgeConv(16,8, batch_norm=True)
        )

        self.edge_reconstruct3 = dglSequential(
            EdgeConv(128, 32, batch_norm=True),
            EdgeConv(32,16, batch_norm=True),
            EdgeConv(16,8, batch_norm=True)
        )

    def forward(self, graph, n_feat=None):
        
        if n_feat is None:
            n_feat = graph.ndata['l']

        # shared path
        shared = self.shared_path(graph, n_feat)

        # node reconstruction
        n = self.node_reconstruct(graph, shared)
        
        # edges reconstruction
        e1 = self.edge_reconstruct1(graph, shared)
        e2 = self.edge_reconstruct2(graph, shared)
        e3 = self.edge_reconstruct3(graph, shared)

        # inner product o matmul?
        e1 = torch.inner(e1, e1)  #their elements are A_{ij}
        e2 = torch.inner(e2, e2) 
        e3 = torch.inner(e3, e3)

        return n, torch.stack([e1, e2, e3], 2)
    

In [44]:
decoder = Decoder(4)
decoded = decoder(batch)

NameError: name 'batch' is not defined

In [ ]:
autoencoder = dglSequential(encoder, decoder)
n, e1, e2, e3 = autoencoder(batch)

## Training

In [8]:
### Initialize the two networks
encoded_space_dim = 6
encoder = Encoder(latent_space_dim=encoded_space_dim)
decoder = Decoder(latent_space_dim=encoded_space_dim)

autoencoder = dglSequential(encoder, decoder)

### Learning Rate and Loss customize 

In [9]:
def custom_loss(output, target):
    """ pred and real are two tuples which contains:
    real: real nodes and edges
    real: real nodes and edges"""

    # regularization parameters    
    l_node = 0.3
    l_edge = 1
    
    loss_fn = torch.nn.MSELoss()
    node_loss = torch.sqrt(loss_fn(output[0], target[0])) 
    e0_loss = torch.sqrt(loss_fn(output[1], target[1])) 
    e1_loss = torch.sqrt(loss_fn(output[2], target[2]))
    e2_loss = torch.sqrt(loss_fn(output[3], target[3]))  

    return l_node * node_loss + l_edge * (e0_loss+e1_loss+e2_loss)


def nodes_loss(output, target):

    loss_fn = torch.nn.MSELoss()
    return torch.sqrt(loss_fn(output, target))   

def edges_loss(output, target):
    #dim = int(np.sqrt(target.shape[0]))
    loss_fn = torch.nn.MSELoss(reduction='none')
    edge_loss = torch.sum(torch.sqrt(torch.mean(loss_fn(output, target), dim=[0, 1])))

    return edge_loss

In [10]:
### Define an optimizer (both for the encoder and the decoder!)
lr = 1e-3 # Learning rate
loss_fn = custom_loss
params_to_optimize = [
    {'params': encoder.parameters()},
    {'params': decoder.parameters()}
]
optimizer = torch.optim.Adam(params_to_optimize, lr=lr)

# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

"""
usando questo bisogna fare scheduler.step(val_loss) alla fine di un epoca
"""
lr_scheduler = ReduceLROnPlateau(
    optimizer,
    mode = 'min',
    factor=0.5,
    patience=5,
    cooldown=5,
    min_lr=10e-8, # this should be the minimum value of the lr at which we stop the training
    threshold=1e-2
    )


encoder.to(device)
decoder.to(device)
autoencoder.to(device)

Selected device: cuda


Sequential(
  (0): Encoder(
    (mlp): Sequential(
      (0): Linear(in_features=3, out_features=256, bias=True)
      (1): Dropout(p=0.2, inplace=False)
      (2): ReLU()
      (3): Linear(in_features=256, out_features=128, bias=True)
      (4): Dropout(p=0.2, inplace=False)
      (5): ReLU()
      (6): Linear(in_features=128, out_features=64, bias=True)
      (7): Dropout(p=0.2, inplace=False)
      (8): ReLU()
      (9): Linear(in_features=64, out_features=32, bias=True)
      (10): Dropout(p=0.2, inplace=False)
      (11): ReLU()
      (12): Linear(in_features=32, out_features=896, bias=True)
      (13): Sigmoid()
    )
    (conv): Sequential(
      (0): NNConv(
        (edge_func): Sequential(
          (0): Linear(in_features=3, out_features=256, bias=True)
          (1): Dropout(p=0.2, inplace=False)
          (2): ReLU()
          (3): Linear(in_features=256, out_features=128, bias=True)
          (4): Dropout(p=0.2, inplace=False)
          (5): ReLU()
          (6): Linear(in

### training and validate function

In [113]:
### Training function
def train_epoch(encoder, decoder, device, dataloader, loss_fn, optimizer):
    # Set train mode for both the encoder and the decoder
    encoder.train()
    decoder.train()

    # abbiamo già definito l'optimizer nella cella precedente
    
    losses = []
    pbar = tqdm(dataloader, 'Steps')
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for batch in pbar: # with "_" we just ignore the labels (the second element of the dataloader tuple)
        
        batch = batch.to(device)
        #encode
        y_encoder_pred = encoder(batch)
        #update graph node features with their latent space representation
        batch.ndata['l'] = y_encoder_pred
        #decode
        y_decoder_pred = decoder(batch)

        dim = int(np.sqrt(batch.edata['d'].shape[0]))
        loss = loss_fn(y_decoder_pred, [batch.ndata['f']] + [batch.edata['d'][:,i].reshape((dim, dim)) for i in range(3)])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.detach().cpu().numpy())
        pbar.set_postfix_str(f'loss: {losses[-1]:.2f}')
        
    losses = np.mean(losses)
    return losses

In [11]:
### Training function
def train_epoch2(autoencoder, device, dataloader, loss_fn, optimizer):
    # Set train mode for both the encoder and the decoder

    autoencoder.train()

    # abbiamo già definito l'optimizer nella cella precedente
    
    losses = []
    l_node = 0.3
    l_edge = 1
    pbar = tqdm(dataloader, 'Steps')
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for batch in pbar: # with "_" we just ignore the labels (the second element of the dataloader tuple)
        
        batch = batch.to(device)

        #decode
        pred_node, pred_edge = autoencoder(batch)

        node_loss = nodes_loss(pred_node, batch.ndata['f'])

        # get target adj matrix
        sparse_coalesce = batch.adj().coalesce()
        indices = sparse_coalesce.indices().to(device)
        size = sparse_coalesce.size() + (3,)
        values = batch.edata['d']

        target_adj = torch.sparse_coo_tensor(indices, values, size).to_dense()
        edge_loss = edges_loss(pred_edge, target_adj)

        del target_adj, indices
        loss = l_node * node_loss + l_edge * edge_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.detach().cpu().numpy())
        pbar.set_postfix_str(f'loss: {np.mean(losses):.2f}')
        
    losses = np.mean(losses)
    return losses

In [12]:
### Training function
def val_epoch2(autoencoder, device, dataloader, loss_fn):
    # Set train mode for both the encoder and the decoder

    autoencoder.eval()

    # abbiamo già definito l'optimizer nella cella precedente
    
    losses = []
    l_node = 0.3
    l_edge = 1
    pbar = tqdm(dataloader, 'Val Steps')
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    with torch.no_grad():
        for batch in pbar: # with "_" we just ignore the labels (the second element of the dataloader tuple)
            
            batch = batch.to(device)

            #decode
            pred_node, pred_edge = autoencoder(batch)

            node_loss = nodes_loss(pred_node, batch.ndata['f'])

            # get target adj matrix
            sparse_coalesce = batch.adj().coalesce()
            indices = sparse_coalesce.indices().to(device)
            size = sparse_coalesce.size() + (3,)
            values = batch.edata['d']

            target_adj = torch.sparse_coo_tensor(indices, values, size).to_dense()
            edge_loss = edges_loss(pred_edge, target_adj)
            del target_adj, indices
            loss = l_node * node_loss + l_edge * edge_loss
            
            losses.append(loss.detach().cpu().numpy())
            pbar.set_postfix_str(f'loss: {np.mean(losses):.2f}')
            
    losses = np.mean(losses)
    return losses

In [118]:
### Testing function
def test_epoch(encoder, decoder, device, dataloader, loss_fn):
    # Set evaluation mode for encoder and decoder
    encoder.eval()
    decoder.eval()

    with torch.no_grad(): # No need to track the gradients
        # Define the lists to store the outputs for each batch
        conc_out = []
        conc_label = []
        for image_batch, _ in dataloader:
            # Move tensor to the proper device
            image_batch = image_batch.to(device)
            # Encode data
            encoded_data = encoder(image_batch)
            # Decode data
            decoded_data = decoder(encoded_data)
            # Append the network output and the original image to the lists
            conc_out.append(decoded_data.cpu())
            conc_label.append(image_batch.cpu())
        # Create a single tensor with all the values in the lists
        conc_out = torch.cat(conc_out)
        conc_label = torch.cat(conc_label) 
        # Evaluate global loss
        val_loss = loss_fn(conc_out, conc_label)
    return val_loss.data

In [13]:
# train the model
epoch = 0
train_losses = []
val_losses = []
while(True):
    # train for one epoch
    print('EPOCH %d' % (epoch + 1))

    '''train_loss = train_epoch(
        encoder=encoder, 
        decoder=decoder, 
        device=device, 
        dataloader=train_dataloader, 
        loss_fn=loss_fn, 
        optimizer=optimizer)
    print(f'TRAIN - EPOCH {epoch+1} - loss: {train_loss}')'''

    train_loss = train_epoch2(
        autoencoder=autoencoder, 
        device=device, 
        dataloader=train_dataloader, 
        loss_fn=loss_fn, 
        optimizer=optimizer)
    print(f'TRAIN - EPOCH {epoch+1} - loss: {train_loss}')

    train_losses.append(train_loss)

    # evaluate the model
    val_loss = val_epoch2(
        autoencoder=autoencoder, 
        device=device, 
        dataloader=val_dataloader, 
        loss_fn=loss_fn)
    # Print Validationloss
    print(f'VALIDATION - EPOCH {epoch+1} - loss: {val_loss}\n')

    val_losses.append(val_loss)

    # schedule the learning rate
    epoch +=1
    lr_scheduler.step(val_loss)

    if optimizer.param_groups[0]['lr'] < 1e-8:
        break

EPOCH 1


Steps:   0%|          | 0/3125 [00:00<?, ?it/s]

TRAIN - EPOCH 1 - loss: 1.0813195705413818


Val Steps:   0%|          | 0/938 [00:00<?, ?it/s]

VALIDATION - EPOCH 1 - loss: 0.6236509680747986

EPOCH 2


...

TRAIN - EPOCH 67 - loss: 0.5642515420913696


Val Steps:   0%|          | 0/938 [00:00<?, ?it/s]

VALIDATION - EPOCH 67 - loss: 0.6127169728279114

EPOCH 68


Steps:   0%|          | 0/3125 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Save model

In [ ]:
model_name = "_50ep64batch"
torch.save(encoder, 'models/encoder' + model_name + '.pkl')
torch.save(decoder, 'models/decoder' + model_name + '.pkl')
torch.save(autoencoder, 'models/autoencoder' + model_name + '.pkl')

torch.save(autoencoder.state_dict(), 'models/autoenc_sd' + model_name + '.pkl')